# Model to predict AirBnB prices

## Get Data from DB

In [1]:
import pandas as pd
import psycopg2 as ps
from sqlalchemy import create_engine

In [2]:
# DB CONFIG
db_name = "adsproject"
db_user = "ads"
db_password = "admin123!"
db_host = "postgres"
db_port = "5432"

table_name = 'airbnb'
table_name_sentiment = "airbnb_sentiment"

In [3]:
def read_table_to_dataframe(db_name, db_user, db_password, db_host, db_port, table_name):
    # Establish a connection to the PostgreSQL database
    conn = ps.connect(database=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
    # Read the table from the PostgreSQL database into a DataFrame
    df = pd.read_sql(f'SELECT * FROM {table_name};', conn) # type: ignore
    # Close the connection
    conn.close()
    return df

In [4]:
# or read from data
# df = pd.read_csv("../data/airbnb_rental_prices_combined_sentiment.csv", sep=";")


In [5]:
# Call the function to read the table from the PostgreSQL database into a DataFrame
df = read_table_to_dataframe(db_name, db_user, db_password, db_host, db_port, table_name_sentiment)

# Display the DataFrame
df.head()

OperationalError: could not translate host name "postgres" to address: nodename nor servname provided, or not known


## Feature engineering

In [6]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()

,name,price,price_reduced,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,...,rating_communication,rating_cleanliness,rating_data_accuracy,rooms,price_y,district,mean_rental_price,Subjectivity_median,Polarity_median,comments_list
0,Serviced Studio Zürich City #B,44.0,0.0,2,1,1,1.0,47.40581,8.55173,36,...,0.0,0.0,0.0,2,44.0,Unterstrass,1522.0,0.0,0.450000,['Zum Übernachten optimal für eine Person.\nFü...
1,2.5 Zimmerwohnung mit Garten,40.0,0.0,2,1,1,1.0,47.36336,8.46183,0,...,0.0,0.0,0.0,2,40.0,Albisrieden,1573.0,NaN,NaN,NaN
2,Spannende Wohnung in Zürich Örlikon,62.0,0.0,2,1,1,1.0,47.40388,8.54967,3,...,5.0,4.7,5.0,2,62.0,Unterstrass,1522.0,0.0,0.478571,['Das Haus ist perfekt an das Zentrum von Züri...
3,Geschätzt einfache Wohnung,66.0,0.0,2,1,1,1.0,47.40140,8.49900,0,...,0.0,0.0,0.0,2,66.0,Höngg,1518.0,NaN,NaN,NaN
4,leider die Wohnung ist nicht verfügbar,64.0,0.0,2,1,1,0.0,47.42600,8.54490,0,...,0.0,0.0,0.0,2,64.0,Seebach,1445.0,NaN,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  701 non-null    object 
 1   price                 701 non-null    float64
 2   price_reduced         701 non-null    float64
 3   guests                701 non-null    int64  
 4   bedrooms              701 non-null    int64  
 5   beds                  701 non-null    int64  
 6   bathrooms             701 non-null    float64
 7   lat                   701 non-null    float64
 8   lng                   701 non-null    float64
 9   nr_of_ratings         701 non-null    int64  
 10  rating_overall        701 non-null    float64
 11  rating_valueformoney  701 non-null    float64
 12  rating_location       701 non-null    float64
 13  rating_checkin        701 non-null    float64
 14  rating_communication  701 non-null    float64
 15  rating_cleanliness    7

In [8]:
df.describe()

,price,price_reduced,guests,bedrooms,beds,bathrooms,lat,lng,nr_of_ratings,rating_overall,...,rating_location,rating_checkin,rating_communication,rating_cleanliness,rating_data_accuracy,rooms,price_y,mean_rental_price,Subjectivity_median,Polarity_median
count,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,...,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,701.000000,579.000000,563.000000,563.000000
mean,191.808845,16.764622,3.275321,1.386591,1.844508,1.110556,47.377550,8.534902,37.099857,3.413766,...,3.406419,3.451070,3.439943,3.377175,3.410556,3.275321,188.155492,1904.746114,0.010001,0.603445
std,114.123119,61.903706,1.587389,0.707343,1.450148,0.212744,0.019654,0.024217,69.017938,2.146085,...,2.190786,2.217441,2.210896,2.176975,2.194058,1.587389,111.305577,385.273322,0.035457,0.217108
min,40.000000,0.000000,2.000000,1.000000,1.000000,0.000000,47.316160,8.460970,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,40.000000,1089.000000,0.000000,-0.212500
25%,120.000000,0.000000,2.000000,1.000000,1.000000,1.000000,47.365010,8.518410,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,119.000000,1522.000000,0.000000,0.481667
50%,153.000000,0.000000,3.000000,1.000000,1.000000,1.000000,47.375200,8.535990,10.000000,4.670000,...,4.700000,4.800000,4.800000,4.700000,4.800000,3.000000,150.000000,1830.000000,0.000000,0.597619
75%,226.000000,0.000000,4.000000,2.000000,2.000000,1.000000,47.387350,8.549840,38.000000,4.890000,...,4.900000,5.000000,5.000000,4.900000,4.900000,4.000000,223.000000,2179.000000,0.000000,0.750000
max,700.000000,581.000000,12.000000,5.000000,25.000000,1.500000,47.432640,8.620730,547.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,12.000000,700.000000,3088.000000,0.333333,1.000000


In [9]:
df.isnull().sum()

name                      0
price                     0
price_reduced             0
guests                    0
bedrooms                  0
beds                      0
bathrooms                 0
lat                       0
lng                       0
nr_of_ratings             0
rating_overall            0
rating_valueformoney      0
rating_location           0
rating_checkin            0
rating_communication      0
rating_cleanliness        0
rating_data_accuracy      0
rooms                     0
price_y                   0
district                  0
mean_rental_price       122
Subjectivity_median     138
Polarity_median         138
comments_list           138
dtype: int64

In [10]:
df[["Polarity_median", "Subjectivity_median", "comments_list", "mean_rental_price"]].describe()

,Polarity_median,Subjectivity_median,mean_rental_price
count,563.000000,563.000000,579.000000
mean,0.603445,0.010001,1904.746114
std,0.217108,0.035457,385.273322
min,-0.212500,0.000000,1089.000000
25%,0.481667,0.000000,1522.000000
50%,0.597619,0.000000,1830.000000
75%,0.750000,0.000000,2179.000000
max,1.000000,0.333333,3088.000000
